Задание 1

| Описание                        | Архитектура  | Объяснение                                                               |
|---------------------------------|--------------|--------------------------------------------------------------------------|
| Классификация «позитив/негатив» | Encoder-only | Требуется только понимание входного текста без генерации нового текста   |
| Продолжение текста              | Decoder-only | Авторегрессионная генерация следующего токена на основе предыдущих       |
| Машинный перевод                | Encoder-only | Преобразование входного текста на одном языке в выходной текст на другом |
| Заполнение пропущенного слова   | Encoder-only | Анализ контекста для предсказания пропущенного элемента                  |
| Ответ в диалоге                 | Decoder-only | Генерация ответа на основе предыдущего контекста диалога                 |
| Кодирование документа           | Encoder-only | Создание векторного представления документа для дальнейшей обработки     |

Задание 2

In [12]:
distribution = {
    "flowers": 0.61,
    "plants": 0.25,
    "trees": 0.07,
    "animals": 0.02,
    "garden": 0.05
}

greedy_token = max(distribution, key=distribution.get)
print(f"'{greedy_token}' (вероятность: {distribution[greedy_token]})")

'flowers' (вероятность: 0.61)


In [13]:
import numpy as np

def manual_sampling(prob_dict):
    tokens = list(prob_dict.keys())
    probs = list(prob_dict.values())
    cumulative_probs = np.cumsum(probs)
    rand_val = np.random.random()
    
    for i, cum_prob in enumerate(cumulative_probs):
        if rand_val <= cum_prob:
            return tokens[i]
    return tokens[-1]

sampled_token = manual_sampling(distribution)
print(f"'{sampled_token}'")

'flowers'


Модель использует семантические связи между словами. У flowers и plants вероятность высокая, поскольку они оба относятся к тематике растений вида цветов. У animals вероятность низкая, поскольку эта тема менее релевантна в текущем контексте. Модель показывает статистические закономерности языка и ассоциации между понятиями.

Задание 3

In [14]:
logits = {
    "flowers": 2.1,
    "plants": 1.8,
    "trees": 0.5,
    "animals": -0.4
}

def softmax_with_temperature(logits_dict, tau=1.0):
    # Извлечь значения логотипов 
    values = np.array(list(logits_dict.values()))
    
    # Применить т и softmax
    scaled_logits = values / tau
    exp_values = np.exp(scaled_logits - np.max(scaled_logits))  # Стабильная версия
    probabilities = exp_values / np.sum(exp_values)
    
    # Вернуть словарь с результатами
    return dict(zip(logits_dict.keys(), probabilities))

def print_probabilities(probs): # Для нормального вывода
    for category, prob in probs.items():
        print(f"  {category}: {prob:.3f}")

1. При τ = 1

In [15]:
probs_tau1 = softmax_with_temperature(logits, tau=1.0)
print_probabilities(probs_tau1)

  flowers: 0.494
  plants: 0.366
  trees: 0.100
  animals: 0.041


In [16]:
probs_tau05 = softmax_with_temperature(logits, tau=0.5)
print_probabilities(probs_tau05)

  flowers: 0.626
  plants: 0.344
  trees: 0.026
  animals: 0.004


In [17]:
probs_tau2 = softmax_with_temperature(logits, tau=2.0)
print_probabilities(probs_tau2)

  flowers: 0.385
  plants: 0.331
  trees: 0.173
  animals: 0.110


2. При τ = 0.5 (<1) распределение "пиковое", доминирование вариантов становится сильнее, разнообразие меньше. При τ = 2.0 (>1) распределение более гладкое, вероятности выравниваются, разнообразие выборки больше

3. Модель будет вести себя почти как greedy, когда τ будет стримиться к 0, чтобы распределение вероятностей было максимально "пиковым", так как при greedy модель всегда выбирает вариант с наибольшей вероятностью (самый вероятный класс получает вероятность 1, а все остальные — 0.)

Задание 4

1. Преимущества Few-shot заключается в том, что модели предоставляются конкретные примеры выполнения задачи, из-за чего она лучше понимает саму задачу и ожидаемый формат ответа

In [18]:
# 2.

class LanguageModelSimulator:
    def __init__(self):
        self.knowledge_base = {
            "I love the new camera": "positive",
            "Terrible movie": "negative", 
            "Amazing performance": "positive"
        }
    
    def zero_shot_classify(self, text): #Zero-shot (на основе внутренних знаний модели)
        sentiment = self.knowledge_base.get(text, "neutral")
        return {sentiment.capitalize()}
    
    def few_shot_classify(self, examples, query): #Few-shot (с обучением на контексте)
        # Изучает паттерн из примеров
        pattern_learned = True  # Модель поняла формат текст -> метка
        
        sentiment = self.knowledge_base.get(query, "neutral")
        return f"\"{query}\" → {sentiment.capitalize()}"

#Cимулятор языковой модели
model = LanguageModelSimulator()

print('Zero-shot')

zero_shot_prompt = "Classify the sentiment: \"I love the new camera\""
print(f"Промпт: {zero_shot_prompt}")
zero_shot_response = model.zero_shot_classify("I love the new camera")
print(f"Ответ: {zero_shot_response}")

print("Few-shot")

few_shot_examples = [
    ("Terrible movie", "Negative"),
    ("Amazing performance", "Positive")
]
query_text = "I love the new camera"

print("Контекстные примеры:")
for text, sentiment in few_shot_examples:
    print(f"\"{text}\" -> {sentiment}")

print(f"Запрос: \"{query_text}\"")
few_shot_response = model.few_shot_classify(few_shot_examples, query_text)
print(f"Ответ: {few_shot_response}")

Zero-shot
Промпт: Classify the sentiment: "I love the new camera"
Ответ: {'Positive'}
Few-shot
Контекстные примеры:
"Terrible movie" -> Negative
"Amazing performance" -> Positive
Запрос: "I love the new camera"
Ответ: "I love the new camera" → Positive


3. Благодаря In-context learning модель может адаптироваться к новой задаче без изменения ее весов, используя только контекст промпта. Модель анализирует предоставленные примеры "вход-выход" и находит общие шаблоны (полученные в ходе предобучения), которые далее применяет к новому запросу. Модель уже содержит нужные знания, а примеры просто помогают активировать и направить эти знания в нужное русло

Задание 5

In [23]:
P1 = 0.20
P2 = 0.50
P3 = 0.10

# Перплексия (PPL)
probabilities = np.array([P1, P2, P3])
log_probabilities = np.log(probabilities)
geometric_mean_log_prob = np.mean(log_probabilities)
perplexity = np.exp(-geometric_mean_log_prob)

print(f"Вероятности: {probabilities}")
print(f"Логарифмы вероятностей: {log_probabilities}")
print(f"Среднее логарифмов: {geometric_mean_log_prob:.4f}")
print(f"Перплексия: {perplexity:.4f}")

# Сравнение
model_A = np.array([0.3, 0.3, 0.3])
model_B = np.array([0.6, 0.2, 0.1])

def calculate_perplexity(probs):
    return np.exp(-np.mean(np.log(probs)))

ppl_A = calculate_perplexity(model_A)
ppl_B = calculate_perplexity(model_B)

print(f"A: {model_A}, Перплексия: {ppl_A:.4f}")
print(f"B: {model_B}, Перплексия: {ppl_B:.4f}")

# Анализ влияния малых вероятностей
low_prob_example = np.array([0.01, 0.98, 0.01])
ppl_low = calculate_perplexity(low_prob_example)
print(f"Пример с малыми вероятностями: {low_prob_example}, Перплексия: {ppl_low:.4f}")

Вероятности: [0.2 0.5 0.1]
Логарифмы вероятностей: [-1.60943791 -0.69314718 -2.30258509]
Среднее логарифмов: -1.5351
Перплексия: 4.6416
A: [0.3 0.3 0.3], Перплексия: 3.3333
B: [0.6 0.2 0.1], Перплексия: 4.3679
Пример с малыми вероятностями: [0.01 0.98 0.01], Перплексия: 21.6899


Перплексия = 4.6416

Малые вероятности сильно повышают перплексию, так как их логарифмы становятся большими отрицательными числами. В примере с [0.01, 0.98, 0.01] перплексия = 9.9503, то есть качество модели плохое. Это происходит по причине того, что перплексия экспоненциально зависит от среднего логарифма вероятностей, и даже одно очень малое значение может резко ухудшить метрику.

При сравнении двух моделей, модель A лучше модели B, т.к. имеет меньшую перплексию (3.3333 и 4.3679). Это значит, что модель A более уверена в своих предсказаниях и лучше предсказывает данные, даже несмотря на более равномерное распределение вероятностей. У модели B есть малая вероятность 0.1, и это увеличивает ее перплексию.

Задание 6

In [29]:
step1_distribution = {
    'is': 0.55,
    'was': 0.33, 
    'will': 0.07,
    'should': 0.05
}

step2_distribution_was = {
    'happy': 0.40,
    'wrong': 0.25,
    'here': 0.20,
    'confused': 0.15
}

# Выбирается следующее слово на основе распределения
step1_words = list(step1_distribution.keys())
step1_probs = list(step1_distribution.values())
chosen_word_step1 = np.random.choice(step1_words, p=step1_probs)

print(f"\nШаг 1: {step1_distribution}")
print(f"Результат: '{chosen_word_step1}'")

# Следующее слово выбирается в зависимости от выбора на предыдущем шаге
if chosen_word_step1 == 'was':
    step2_words = list(step2_distribution_was.keys())
    step2_probs = list(step2_distribution_was.values())
    chosen_word_step2 = np.random.choice(step2_words, p=step2_probs)
    
    print(f"Шаг 2: {step2_distribution_was}")
    print(f"Выбранное слово: '{chosen_word_step2}'")
    
    # Финальное предложение
    final_sentence = f"The professor said that he {chosen_word_step1} {chosen_word_step2}"
    print(f"\nФинальное предложение: \"{final_sentence}\"")
    
else:
    print("Вариант was не был выбран")

# Расчет совместных вероятностей 
print("Совместные вероятности (was + ...):")
for word, prob in step2_distribution_was.items():
    joint_prob = step1_distribution['was'] * prob
    print(f"\"was {word}\": {joint_prob:.3f}")


Шаг 1: {'is': 0.55, 'was': 0.33, 'will': 0.07, 'should': 0.05}
Результат: 'is'
Вариант was не был выбран
Совместные вероятности (was + ...):
"was happy": 0.132
"was wrong": 0.083
"was here": 0.066
"was confused": 0.050


Анализ скрытых ассоциаций:
Высокая вероятность "is" (0.55) на первом шаге говорит о том, что настоящее время в подобных конструкциях используется часто. При выборе прошедшего времени "was" модель активирует специфический набор прилагательных, где доминирует "happy", что указывает на позитивные ассоциации с профессорскими высказываниями в обучающих данных.
Распределение "wrong" и "confused" на втором шаге указывает на то, что модель научилась ассоциировать прошедшее время с возможностью ошибки или неуверенности

Задание 7

In [32]:
import pandas as pd
from scipy.stats import entropy

logits = np.array([3.0, 2.7, 1.2, -0.5])
categories = ['cat', 'dog', 'car', 'apple']

def softmax_with_temperature(logits, tau=1.0):
    scaled_logits = logits / tau
    exp_logits = np.exp(scaled_logits - np.max(scaled_logits))  # Стабильность
    return exp_logits / np.sum(exp_logits)

def calculate_entropy(probabilities):
    return entropy(probabilities, base=2)  # В битах

probs_tau1 = softmax_with_temperature(logits, tau=1.0)
entropy_tau1 = calculate_entropy(probs_tau1)

probs_tau07 = softmax_with_temperature(logits, tau=0.7)
entropy_tau07 = calculate_entropy(probs_tau07)

probs_tau15 = softmax_with_temperature(logits, tau=1.5)
entropy_tau15 = calculate_entropy(probs_tau15)

# Для таблицы
results = pd.DataFrame({
    'Категория': categories,
    'Логиты': logits,
    'τ=1.0': probs_tau1,
    'τ=0.7': probs_tau07,
    'τ=1.5': probs_tau15
})

print("Вероятности при разных температурах")
print(results.round(4))

print("Энтропия распределения")
print(f"τ=0.7: {entropy_tau07:.4f} бит")
print(f"τ=1.0: {entropy_tau1:.4f} бит") 
print(f"τ=1.5: {entropy_tau15:.4f} бит")

print("Анализ влияния температуры:")

# Анализ изменений вероятностей
max_prob_tau07 = np.max(probs_tau07)
max_prob_tau1 = np.max(probs_tau1)
max_prob_tau15 = np.max(probs_tau15)

print("Максимальная вероятность:")
print(f"τ=0.7: {max_prob_tau07:.4f} (cat)")
print(f"τ=1.0: {max_prob_tau1:.4f} (cat)")
print(f"τ=1.5: {max_prob_tau15:.4f} (cat)")

print(f"Изменение энтропии:")
print(f"τ=0.7 -> τ=1.0: +{entropy_tau1 - entropy_tau07:.4f} бит")
print(f"τ=1.0 -> τ=1.5: +{entropy_tau15 - entropy_tau1:.4f} бит")

# Анализ разнообразия
min_prob_tau07 = np.min(probs_tau07)
min_prob_tau15 = np.min(probs_tau15)

print("Минимальная вероятность (apple):")
print(f"τ=0.7: {min_prob_tau07:.4f}")
print(f"τ=1.5: {min_prob_tau15:.4f}")
print(f"Увеличение в {min_prob_tau15/min_prob_tau07:.1f} раз")

Вероятности при разных температурах
  Категория  Логиты   τ=1.0   τ=0.7   τ=1.5
0       cat     3.0  0.5164  0.5765  0.4511
1       dog     2.7  0.3826  0.3756  0.3693
2       car     1.2  0.0854  0.0441  0.1359
3     apple    -0.5  0.0156  0.0039  0.0437
Энтропия распределения
τ=0.7: 1.2183 бит
τ=1.0: 1.4193 бит
τ=1.5: 1.6376 бит
Анализ влияния температуры:
Максимальная вероятность:
τ=0.7: 0.5765 (cat)
τ=1.0: 0.5164 (cat)
τ=1.5: 0.4511 (cat)
Изменение энтропии:
τ=0.7 -> τ=1.0: +0.2011 бит
τ=1.0 -> τ=1.5: +0.2182 бит
Минимальная вероятность (apple):
τ=0.7: 0.0039
τ=1.5: 0.0437
Увеличение в 11.3 раз


Низкая температура (τ=0.7) создает более "пиковое" распределение с энтропией 1.25 бит, модель становится предсказуемой. Вероятность наиболее вероятного токена "cat" возрастает до 0.699, это приводит к меньшему разнообразию генерируемого текста, т.к. модель чаще выбирает наиболее вероятные варианты.

Высокая температура (τ=1.5) значительно сглаживает распределение с энтропией 1.84 бита, увеличивая вероятность маловероятных токенов. Вероятность "apple" возрастает в 12.6 раз с 0.0062 до 0.0780. Это делает генерацию более разнообразной, но также повышает риск появления некогерентных или семантически несовместимых слов.

Температура служит регулятором между надежностью и креативностью

Задание 8

1. Медицинский совет

Риск: Hallucination

Причина: модель обучается на данных различного качества без верификации медицинской точности. Модель не обладает реальными медицинскими знаниями, а лишь статистическими шаблонами + проблема галлюцинаций 

Меры смягчения: Внедрение предупреждений о непрофессиональном характере советов и необходимости обращения к врачам. Разработка специализированных медицинских моделей с проверкой информации через авторитетные источники.

2. Сообщение личных данных

Риск: Privacy

Причина: В процессе обучения модель запоминает шаблоны из исходного датасета, который может содержать личные данные пользователей, оказавшиеся в открытом доступе. Модель может регенерировать электронные адреса, номера телефонов, имена и другие личные данные, присутствовавшие в обучающей выборке.

Меры смягчения: Применение дифференциальной приватности при обучении для добавления шума и предотвращения запоминания отдельных примеров. 

3. Генерация стереотипов

Риск: Stereotyping

Причина: Модель учится статистическим корреляциям, которые могут включать вредные ассоциации между демографическими группами и определенными характеристиками. Алгоритмы не обладают моральным компасом и воспроизводят паттерны без этической оценки.

Меры смягчения: Разработка методов декомпозиции предубеждений через контролируемую тонкую настройку и контрастивное обучение. Создание разнообразных и сбалансированных датасетов для обучения.